In [1]:
import pandas as pd

# Cargar el archivo original
input_file = "/Users/sandra/Downloads/FinalResults_gpt41nano.xlsx"
output_file = "MMLU/paraphrases_ChatGPT-41-nano.xlsx"

# Leer el archivo eliminando la primera columna sin nombre
df = pd.read_excel(input_file, index_col=0)

# Renombrar las columnas correctamente
df = df.rename(columns={
    df.columns[0]: "question", df.columns[1]: "A", df.columns[2]: "B",
    df.columns[3]: "C", df.columns[4]: "D", df.columns[5]: "correct_answer",
    df.columns[6]: "llm_answer_filtered",
    df.columns[7]: "question", df.columns[8]: "A", df.columns[9]: "B",
    df.columns[10]: "C", df.columns[11]: "D",
    df.columns[12]: "llm_answer_filtered"
})

# Crear las dos hojas separadas
df_en_l0 = df.iloc[:, :7]  # Columnas 0 a 6
df_en_l1 = df.iloc[:, [7, 8, 9, 10, 11, 5, 12]]  # Columnas 7 a 12, con correct_answer y llm_answer_filtered correctos

# Guardar en un nuevo archivo Excel sin la columna "Unnamed: 0"
with pd.ExcelWriter(output_file) as writer:
    df_en_l0.to_excel(writer, sheet_name="en_l0", index=False)
    df_en_l1.to_excel(writer, sheet_name="en_l1", index=False)

print(f"Archivo procesado y guardado como {output_file}")



Archivo procesado y guardado como MMLU/paraphrases_ChatGPT-41-nano.xlsx


In [6]:
import pandas as pd

# Cargar el archivo original
input_file = "MMLU_pro/ChatGPT-MMLUpro.xlsx"
output_file = "MMLU_pro/paraphrases_ChatGPT-4o-mini.xlsx"

# Leer el archivo sin eliminar ninguna columna
df = pd.read_excel(input_file)

# Renombrar las columnas correctamente
column_names = {
    0: "question",
    1: "A", 2: "B", 3: "C", 4: "D", 5: "E", 6: "F", 7: "G", 8: "H", 9: "I", 10: "J",
    11: "correct_answer",
    12: "llm_answer_filtered",
    13: "question",
    14: "A", 15: "B", 16: "C", 17: "D", 18: "E", 19: "F", 20: "G", 21: "H", 22: "I", 23: "J",
    24: "llm_answer_filtered"
}
df = df.rename(columns=column_names)

# Crear las dos hojas separadas
df_en_l0 = df.iloc[:, :13]  # Columnas 0 a 12
df_en_l1 = df.iloc[:, [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 11, 24]]  # Columnas 13 a 24, con correct_answer (col 11)

# Guardar en un nuevo archivo Excel
with pd.ExcelWriter(output_file) as writer:
    df_en_l0.to_excel(writer, sheet_name="en_l0", index=False)
    df_en_l1.to_excel(writer, sheet_name="en_l1", index=False)

print(f"Archivo procesado y guardado como {output_file}")

Archivo procesado y guardado como MMLU_pro/paraphrases_ChatGPT-4o-mini.xlsx


In [7]:
import pandas as pd
import json
import re

def extraer_datos(json_str):
    """Extrae la pregunta y opciones de respuesta de un JSON en formato string."""
    try:
        # Extraemos solo el JSON ignorando cualquier otro texto adicional
        match = re.search(r'\{.*\}', json_str)
        if match:
            json_str = match.group()
        else:
            return None
        
        data = json.loads(json_str)
        question = data.get("question", "")
        
        options = {chr(65+i): data.get(chr(65+i), "-") for i in range(10)}  # A-J
        
        return [question] + list(options.values())
    except Exception as e:
        return None

def procesar_excel(archivo_excel, archivo_ids, output_excel):
    # Cargar archivo principal
    df = pd.read_excel(archivo_excel, sheet_name='en_l1')
    df_ids = pd.read_excel(archivo_ids)  # Archivo con IDs
    
    print(df_ids.head())  # Verifica las primeras filas del archivo de IDs    
    # Extraer datos
    columnas = ['instruction', 'option_a', 'option_b', 'option_c', 'option_d', 'option_e', 'option_f', 'option_g', 'option_h', 'option_i', 'option_j']
    datos_extraidos = df['raw_paraphrase'].dropna().apply(extraer_datos)
    
    # Crear DataFrame con los datos extraídos
    df_limpio = pd.DataFrame(datos_extraidos.tolist(), columns=columnas)
    
    # Añadir la columna ID desde el otro archivo
    df_limpio['id'] = df_ids.iloc[:, 0]  # Copia la primera columna de df_ids
    
    # Guardar el resultado en un nuevo archivo Excel
    df_limpio.to_excel(output_excel, index=False)
    print(f"Archivo guardado como {output_excel}")

# Parámetros
archivo_excel = "paraphrases_gemma-2-9b-it_16bits_EN-4.xlsx"  # Reemplaza con la ruta correcta
archivo_ids = "../Datasets/MMLU_pro_completo.xlsx"  # Reemplaza con la ruta correcta
output_excel = "gemma.xlsx"

procesar_excel(archivo_excel, archivo_ids, output_excel)


                                         instruction  \
0  Typical advertising regulatory bodies suggest,...   
1  Managers are entrusted to run the company in t...   
2  There are two main issues associated with ____...   
3  _______ locate morality beyond the sphere of r...   
4   Some of key differences between Islamic finan...   

                                            option_a  \
0            Safe practices, Fear, Jealousy, Trivial   
1             Shareholders, Diligence, Self-interest   
2              Down, Autonomy, Remuneration, Benefit   
3                                     Ethical egoism   
4  Interest, Certain, Assured, Both tangible and ...   

                                            option_b  \
0           Unsafe practices, Distress, Joy, Trivial   
1        Shareholders, Self-interest, Care and Skill   
2           Down, Involvement, Independence, Benefit   
3                                     Ethics of duty   
4  Interest, Uncertain, Assured, Both tangible

TypeError: object of type 'NoneType' has no len()

In [18]:
import pandas as pd
import json
import re

def clean_text(text):
    """Elimina caracteres fuera del rango ASCII imprimible y espacios adicionales."""
    if isinstance(text, str):
        text = re.sub(r'[^\x20-\x7E]', '', text)  # Elimina caracteres no imprimibles
        text = re.sub(r'\s+', ' ', text)  # Reemplaza múltiples espacios por uno solo
        return text.strip()
    return text

def fix_json_format(json_text):
    """Corrige errores comunes en el JSON."""
    json_text = json_text.replace("'", '"')  # Reemplazar comillas simples por dobles
    json_text = re.sub(r'(\w)\s*:\s*', r'\1: ', json_text)  # Eliminar espacios innecesarios
    json_text = re.sub(r'(?<!")(\b\w+\b)(?=\s*:)', r'"\1"', json_text)  # Asegurar comillas en claves
    return json_text

def extract_json(text):
    """Extrae el contenido JSON eliminando cualquier otro texto alrededor."""
    # Buscar el bloque JSON dentro del texto
    match = re.search(r'\{.*\}', text, re.DOTALL)
    if match:
        return match.group()
    return None  # Retorna None si no encuentra un JSON válido

archivo_excel = "paraphrases_gemma-2-9b-it_16bits_EN-4.xlsx"
archivo_ids = "../Datasets/MMLU_pro_completo.xlsx"

# Leer los archivos
df = pd.read_excel(archivo_excel, sheet_name="en_l1")
df_ids = pd.read_excel(archivo_ids)

# Verificar que los archivos tengan la misma cantidad de filas
if len(df) != len(df_ids):
    raise ValueError("Los archivos tienen diferente cantidad de filas.")

data = []

# Recorrer cada fila del dataframe
for i, row in df.iterrows():
    raw_text = row.get("raw_paraphrase", "")
    answer = row.get("answer", "-")  # Respuesta correcta

    # Extraer solo el JSON desde el texto
    json_text = extract_json(raw_text)
    
    if not json_text:
        # Si no hay JSON válido, se agrega un registro con valores por defecto
        data.append(["Pregunta no válida"] + ["-"] * 10 + [df_ids.loc[i, 'id'], answer])
        continue

    try:
        # Intentar cargar el JSON
        parsed_json = json.loads(json_text)
    except json.JSONDecodeError:
        # Intentar limpiar y corregir el JSON
        json_text = clean_text(json_text)
        json_text = fix_json_format(json_text)
        try:
            parsed_json = json.loads(json_text)
        except json.JSONDecodeError:
            # Si sigue fallando, marcar la pregunta como inválida
            question = "Pregunta no válida"
            options = {key.lower(): "-" for key in "ABCDEFGHIJ"}
            data.append([question] + list(options.values()) + [df_ids.loc[i, 'id'], answer])
            continue

    # Extraer pregunta y opciones
    question = clean_text(parsed_json.get("question", "Pregunta no válida").strip())
    options = {key.lower(): clean_text(parsed_json.get(key, "-")) for key in "ABCDEFGHIJ"}

    # Agregar datos a la lista
    data.append([question] + [options[f"{chr(97 + j)}"] for j in range(10)] + [df_ids.loc[i, 'id']] + [df_ids.loc[i, 'answer']])

# Crear DataFrame con las columnas especificadas
columnas = ["instruction", "option_a", "option_b", "option_c", "option_d", 
            "option_e", "option_f", "option_g", "option_h", "option_i", "option_j", "id", "answer"]

df_final = pd.DataFrame(data, columns=columnas)

# Guardar el resultado en un nuevo archivo Excel
df_final.to_excel("resultado.xlsx", index=False, engine="openpyxl")

print("Proceso completado. Archivo guardado como 'resultado.xlsx'.")


Proceso completado. Archivo guardado como 'resultado.xlsx'.


In [2]:
import pandas as pd

# Cargar el archivo procesado previamente
input_file = "MMLU/paraphrases_ChatGPT-41-nano.xlsx"
output_file = "MMLU/paraphrases_ChatGPT-41-nano.xlsx"

# Cargar las hojas en_l0 y en_l1
df_en_l0 = pd.read_excel(input_file, sheet_name="en_l0")
df_en_l1 = pd.read_excel(input_file, sheet_name="en_l1")

# Función para calcular estadísticas
def calcular_estadisticas(df, experiment_name):
    num_correct = (df["correct_answer"] == df["llm_answer_filtered"]).sum()
    total_questions = len(df)
    num_wrong = total_questions - num_correct
    num_null = df["llm_answer_filtered"].isna().sum()
    accuracy = (num_correct / total_questions) * 100 if total_questions > 0 else 0
    
    return [experiment_name, num_correct, accuracy, num_wrong, num_null]

# Calcular estadísticas para en_l0 y en_l1
stats_l0 = calcular_estadisticas(df_en_l0, "en_l0_eval")
stats_l1 = calcular_estadisticas(df_en_l1, "en_l1_eval")

# Crear DataFrame de estadísticas
df_eval_stats = pd.DataFrame([stats_l0, stats_l1], columns=[
    "Experiment", "Number of correct answers", "Accuracy (%)", 
    "Number of wrong answers", "Number of null answers"
])

# Guardar todo en un nuevo archivo Excel
with pd.ExcelWriter(output_file) as writer:
    df_en_l0.to_excel(writer, sheet_name="en_l0", index=False)
    df_en_l1.to_excel(writer, sheet_name="en_l1", index=False)
    df_eval_stats.to_excel(writer, sheet_name="eval_stats", index=False)

print(f"Archivo final guardado como {output_file}")


Archivo final guardado como MMLU/paraphrases_ChatGPT-41-nano.xlsx
